In [1]:
# from google import google
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import MetaData
from configparser import ConfigParser
import os
import spacy
nlp = spacy.load('en')
import re
import phonenumbers
import contextlib
import numpy as np
from sunlight import congress
import sunlight
sunlight.apikey = 'thisisakey'
import math
import datetime

from imwithdata.es_etl.issues_actions import (
    issues,
    actions,
    state_regex,
    city_regex,
    web_url_regex,
    date_include_regex,
    date_exclude_regex,
    leg_name_regex,
    leg_twitter_regex
)

In [2]:
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

mysql_creds = get_ini_vals(config_file, 'mysql')

engine = create_engine("""mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8""".format(user=mysql_creds['user'],
                                                                                 password=mysql_creds['password'],
                                                                                 host=mysql_creds['host'],
                                                                                 port=mysql_creds['port'],
                                                                                 db=mysql_creds['database'], encoding="utf-8"
                                                                                )
                      )
conn = engine.connect()


In [4]:
meetup_data = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/meetup_test.csv')
print(meetup_data.columns)
print(meetup_data['_source.link'].tolist()[:10])


Index(['_id', '_index', '_score', '_source.created', '_source.description',
       '_source.duration', '_source.group.create', '_source.group.created',
       '_source.group.id', '_source.group.join_mode', '_source.group.lat',
       '_source.group.lon', '_source.group.name', '_source.group.urlname',
       '_source.group.who', '_source.how_to_find_us', '_source.id',
       '_source.link', '_source.manual_attendance_count', '_source.name',
       '_source.rsvp_limit', '_source.status', '_source.time',
       '_source.updated', '_source.utc_offset', '_source.venue.address_1',
       '_source.venue.city', '_source.venue.country', '_source.venue.id',
       '_source.venue.lat', '_source.venue.localized_country_name',
       '_source.venue.lon', '_source.venue.name', '_source.venue.phone',
       '_source.venue.repinned', '_source.venue.state', '_source.venue.zip',
       '_source.visibility', '_source.waitlist_count',
       '_source.yes_rsvp_count', '_type'],
      dtype='object')
['http

In [5]:
# print (congress.locate_districts_by_lat_lon(41.92, -80.14))
# print (congress.locate_districts_by_zip(52245))
# print (congress.locate_legislators_by_zip(52245))

associated_legislators = []
associated_districts = []

for i, lat in enumerate(meetup_data['_source.group.lat'].tolist()):
    if lat:
        legs = '; '.join([leg['title'] + '. ' + leg['first_name'] + ' ' + leg['last_name'] 
                          for leg in 
                          congress.locate_legislators_by_lat_lon(lat, meetup_data['_source.group.lon'].tolist()[i])])
        dists = '; '.join([dist['state'] + '-' + str(dist['district']).zfill(2) 
                 for dist in 
                 congress.locate_districts_by_lat_lon(lat,meetup_data['_source.group.lon'].tolist()[i])])
    else:
        legs = ''
        dists = ''
        
    associated_legislators.append(legs)
    associated_districts.append(dists)
    
    
        


In [6]:
meetup_data['legislators'] = associated_legislators
meetup_data['districts'] = associated_districts

In [54]:
[dist['state'] + '-' + str(dist['district']).zfill(2) for dist in congress.locate_districts_by_zip(52245)]

['IA-02']

In [75]:

# meetup_data['date'] = (pd.to_datetime(meetup_data['_source.time']) - meetup_data['timedelt']).dt.strftime('%Y-%m-%d')

# print(meetup_data['date'][:10])
# print(meetup_data['_source.name'][:10])


In [7]:
meetup_data['timedelt'] = pd.Series([ pd.Timedelta(milliseconds=i) for i in meetup_data['_source.utc_offset'].tolist()])
meetup_data['event_delt'] = pd.Series([ pd.Timedelta(milliseconds=i) 
                                       if math.isnan(i) == False else 0.0 
                                       for i in meetup_data['_source.duration'].tolist()])


meetup_data['event_id'] = meetup_data['_id']
meetup_data['event_source'] = 'Meetup.com'
meetup_data['event_score'] = meetup_data['_score'] * 100
meetup_data['event_issues'] = meetup_data['_index']
meetup_data['event_title'] = meetup_data['_source.name']
meetup_data['event_description'] = meetup_data['_source.description'].str.encode('utf-8')
meetup_data['event_location_name'] = meetup_data['_source.venue.name']
meetup_data['event_address'] = meetup_data['_source.venue.address_1']
meetup_data['event_city'] = meetup_data['_source.venue.city']
meetup_data['event_state'] = meetup_data['_source.venue.state']
meetup_data['event_zip'] = meetup_data['_source.venue.zip']
meetup_data['event_district'] = meetup_data['districts']
meetup_data['event_full_address'] = meetup_data['event_location_name'].map(str) + ', ' + meetup_data['event_address'].map(str) + ', ' + meetup_data['event_city'].map(str) + ', ' + meetup_data['event_state'].map(str) + ' ' + meetup_data['event_zip'].map(str) 
meetup_data['event_location_phone'] = meetup_data['_source.venue.phone']
meetup_data['event_rsvp_to'] = meetup_data['_source.link']
meetup_data['event_lat'] = meetup_data['_source.venue.lat']
meetup_data['event_lng'] = meetup_data['_source.venue.lon']
meetup_data['event_date'] = (pd.to_datetime(meetup_data['_source.time']) - meetup_data['timedelt']).dt.strftime('%Y-%m-%d')
meetup_data['event_start_time'] = (pd.to_datetime(meetup_data['_source.time']) - meetup_data['timedelt']).dt.strftime('%H:%M')
meetup_data['event_end_time'] = (pd.to_datetime(meetup_data['_source.time']) + meetup_data['event_delt']).dt.strftime('%H:%M')
meetup_data['event_time_zone'] = ''
meetup_data['event_url'] = meetup_data['_source.link']
meetup_data['event_group_associated'] =  meetup_data['_source.group.name']
meetup_data['event_group_url'] = '<p><a href="https://www.meetup.com/" target="_blank">Meetup.com</a></p>'
meetup_data['event_legislator'] = meetup_data['legislators']
meetup_data['event_meeting_type'] = 'Meetup'

In [8]:
meetup_final = meetup_data[[
                'event_id',
                'event_source',
                'event_score',
                'event_issues',
                'event_title',
                'event_description',
                'event_location_name',
                'event_address',
                'event_city',
                'event_state',
                'event_zip',
                'event_district',
                'event_full_address',
                'event_location_phone',
                'event_rsvp_to',
                'event_lat',
                'event_lng',
                'event_date',
                'event_start_time',
                'event_end_time',
                'event_time_zone',
                'event_url',
                'event_group_associated',
                'event_group_url',
                'event_legislator',
                'event_meeting_type'
                      ]]

In [13]:
townhalls = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/THP 4-3.csv')
townhalls.columns


Index(['|', '|__City', '|__Date', '|__District', '|__Location', '|__Member',
       '|__Notes', '|__Party', '|__RSVP', '|__State', '|__StateAb', '|__Time',
       '|__Zip', '|__address', '|__dateChanged', '|__dateObj', '|__dateString',
       '|__dateValid', '|__eventId', '|__eventName', '|__lastUpdated',
       '|__lastUpdatedHuman', '|__lat', '|__link', '|__linkName', '|__lng',
       '|__meetingType', '|__rsvpName', '|__noLoc', '|__repeatingEvent',
       '|__streetAddress', '|__timeEnd', '|__updatedBy', '|__timeEnd24',
       '|__timeStart24', '|__timeZone', '|__yearMonthDay', '|__zoneString'],
      dtype='object')

In [15]:
# townhalls['|']

In [16]:
townhalls['event_id'] = townhalls['|']
townhalls['event_source'] = 'Townhall Project'
townhalls['event_score'] = 65
townhalls['event_issues'] = 'All Issues'
townhalls['event_title'] = townhalls['|__Member'] + ' ' + townhalls['|__meetingType']
townhalls['event_description'] = townhalls['|__Notes'].str.encode('utf-8')
townhalls['event_location_name'] = townhalls['|__Location']
townhalls['event_address'] = townhalls['|__address']
townhalls['event_city'] = townhalls['|__City']
townhalls['event_state'] = townhalls['|__State']
townhalls['event_zip'] = townhalls['|__Zip']
townhalls['event_district'] = townhalls['|__District']
townhalls['event_full_address'] = townhalls['|__Location'].map(str) + ', ' + townhalls['|__address'].map(str) + ', ' + townhalls['|__City'].map(str) + ', ' + townhalls['|__State'].map(str) + ' ' + townhalls['|__Zip'].map(str) 
townhalls['event_location_phone'] = ''
townhalls['event_rsvp_to'] = townhalls['|__RSVP']
townhalls['event_lat'] = townhalls['|__lat']
townhalls['event_lng'] = townhalls['|__lng']
townhalls['event_date'] = pd.to_datetime(townhalls['|__yearMonthDay']).dt.strftime('%Y-%m-%d')
townhalls['event_start_time'] = townhalls['|__timeStart24']
townhalls['event_end_time'] = townhalls['|__timeEnd24']
townhalls['event_time_zone'] = townhalls['|__timeZone']
townhalls['event_url'] = townhalls['|__link']
townhalls['event_group_associated'] = 'Townhall Project'
townhalls['event_group_url'] = 'https://townhallproject.com/'
townhalls['event_legislator'] = townhalls['|__Member']
townhalls['event_meeting_type'] = townhalls['|__meetingType']



In [17]:
townhalls_final = townhalls[['event_id',
                'event_score',
                'event_issues',
                'event_title',
                'event_description',
                'event_location_name',
                'event_address',
                'event_city',
                'event_state',
                'event_zip',
                'event_district',
                'event_full_address',
                'event_location_phone',
                'event_rsvp_to',
                'event_lat',
                'event_lng',
                'event_date',
                'event_start_time',
                'event_end_time',
                'event_time_zone',
                'event_url',
                'event_group_associated',
                'event_group_url',
                'event_legislator',
                'event_meeting_type']]

print(townhalls_final.columns)
print(len(townhalls_final))

Index(['event_id', 'event_score', 'event_issues', 'event_title',
       'event_description', 'event_location_name', 'event_address',
       'event_city', 'event_state', 'event_zip', 'event_district',
       'event_full_address', 'event_location_phone', 'event_rsvp_to',
       'event_lat', 'event_lng', 'event_date', 'event_start_time',
       'event_end_time', 'event_time_zone', 'event_url',
       'event_group_associated', 'event_group_url', 'event_legislator',
       'event_meeting_type'],
      dtype='object')
269


In [18]:
meetup_final.drop_duplicates(inplace=True)
townhalls_final.drop_duplicates(inplace=True)

/Users/brosskatz/anaconda3/lib/python3.6/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [19]:
final_events_table = meetup_final.append(townhalls_final,ignore_index=True)

In [20]:
mask = (pd.to_datetime(final_events_table['event_date'])>datetime.datetime.today())

final_events_table = final_events_table[mask]

In [21]:
final_events_table.to_sql('rzst_events',conn,if_exists='replace',index=False)